In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
class RecommendationSystemSVM:
    def __init__(self):
        self.svm_model = SVC(kernel='linear')
        self.label_encoder = LabelEncoder()
        self.is_trained = False

    def load_data(self, customers_file, products_file, ratings_file):
        with open(customers_file, 'r') as f:
            self.customers_data = json.load(f)

        with open(products_file, 'r') as f:
            self.products_data = json.load(f)

        with open(ratings_file, 'r') as f:
            self.ratings_data = json.load(f)

    def preprocess_data(self):
        customers_df = pd.DataFrame(self.customers_data)
        products_df = pd.DataFrame(self.products_data)
        ratings_df = pd.DataFrame(self.ratings_data)
        customers_df.rename(columns={'Id': 'CustomerID'}, inplace=True)
        products_df.rename(columns={'Id': 'ProductID'}, inplace=True)
        user_item_df = pd.merge(ratings_df, customers_df, on='CustomerID')
        user_item_df = pd.merge(user_item_df, products_df, on='ProductID')

        # Encode categorical features like 'Age', 'Region', 'Category', 'genre'
        label_encoder = LabelEncoder()
        user_item_df['Age_encoded'] = label_encoder.fit_transform(user_item_df['Age'])
        user_item_df['Region_encoded'] = label_encoder.fit_transform(user_item_df['Region'])
        user_item_df['Category_encoded'] = label_encoder.fit_transform(user_item_df['Category'])
        user_item_df['genre_encoded'] = label_encoder.fit_transform(user_item_df['genre'])
        
        # Encode user and product IDs
        user_item_df['user_id_encoded'] = self.label_encoder.fit_transform(user_item_df['CustomerID'])
        user_item_df['product_id_encoded'] = self.label_encoder.fit_transform(user_item_df['ProductID'])

        # Define features and target variable
        features = user_item_df[['user_id_encoded', 'product_id_encoded', 'Age_encoded', 'Region_encoded', 'Category_encoded', 'genre_encoded']]
        target = user_item_df['Rate']

        # Split data into train and test sets
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            features, target, test_size=0.2, random_state=42)

    def train_model(self):
        self.svm_model.fit(self.X_train, self.y_train)
        self.is_trained = True

    def evaluate_model(self):
        if not self.is_trained:
            print("Model is not trained. Call train_model() first.")
            return

        y_pred = self.svm_model.predict(self.X_test)

        accuracy = accuracy_score(self.y_test, y_pred)
        recall = recall_score(self.y_test, y_pred, average='weighted')
        f1 = f1_score(self.y_test, y_pred, average='weighted')
        confusion = confusion_matrix(self.y_test, y_pred)

        print("Accuracy:", accuracy)
        print("Recall:", recall)
        print("F1 Score:", f1)
        print("Confusion Matrix:")
        print(confusion)


In [ ]:
if __name__ == "__main__":
    recommendation_system = RecommendationSystemSVM()
    recommendation_system.load_data('input/dataset/customers.json', 'input/dataset/products.json', 'input/dataset/ratings.json')
    recommendation_system.preprocess_data()
    recommendation_system.train_model()
    recommendation_system.evaluate_model()